In [ ]:
!pip install transformers

In [ ]:
!pip install torch

In [ ]:
!pip install clean-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 6.9 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=8d5bd6073b158285914424fff6ce6a05fd49df19a4c629cd4f76229fa43f4930
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


In [ ]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.4 MB/s eta 0:00:00


In [ ]:
!git clone "https://github.com/cobanov/shakespeare-dataset.git"

Cloning into 'shakespeare-dataset'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 56 (delta 4), reused 3 (delta 0), pack-reused 0
Receiving objects: 100% (56/56), 2.08 MiB | 3.76 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
# Reading the dataset

with open('/content/shakespeare-dataset/text/a-midsummer-nights-dream_TXT_FolgerShakespeare.txt','r',encoding='utf-8') as f:
  text = f.read()

In [ ]:
print("length of text in characters : ",len(text))

length of text in characters :  97213


In [ ]:
#looking the first 1000 characters
print(text[0:1000])

A Midsummer Night's Dream
by William Shakespeare
Edited by Barbara A. Mowat and Paul Werstine
  with Michael Poston and Rebecca Niles
Folger Shakespeare Library
https://shakespeare.folger.edu/shakespeares-works/a-midsummer-nights-dream/
Created on Jul 31, 2015, from FDT version 0.9.2

Characters in the Play
Four lovers:
  HERMIA
  LYSANDER
  HELENA
  DEMETRIUS
THESEUS, duke of Athens
HIPPOLYTA, queen of the Amazons
EGEUS, father to Hermia
PHILOSTRATE, master of the revels to Theseus
NICK BOTTOM, weaver
PETER QUINCE, carpenter
FRANCIS FLUTE, bellows-mender
TOM SNOUT, tinker
SNUG, joiner
ROBIN STARVELING, tailor
OBERON, king of the Fairies
TITANIA, queen of the Fairies
ROBIN GOODFELLOW, a "puck," or hobgoblin, in Oberon's service
A FAIRY, in the service of Titania
Fairies attending upon Titania:
  PEASEBLOSSOM
  COBWEB
  MOTE
  MUSTARDSEED
Lords and Attendants on Theseus and Hippolyta
Other Fairies in the trains of Titania and Oberon


ACT 1
=====

Scene 1



In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !"'(),-./0123459:;=?ABCDEFGHIJKLMNOPQRSTUVWY[]abcdefghijklmnopqrstuvwxyz
75


In [ ]:
#encoding
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s:[stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[0:1000])

torch.Size([97213]) torch.int64
tensor([23,  2, 35, 57, 52, 67, 69, 61, 61, 53, 66,  2, 36, 57, 55, 56, 68,  5,
        67,  2, 26, 66, 53, 49, 61,  1, 50, 73,  2, 45, 57, 60, 60, 57, 49, 61,
         2, 41, 56, 49, 59, 53, 67, 64, 53, 49, 66, 53,  1, 27, 52, 57, 68, 53,
        52,  2, 50, 73,  2, 24, 49, 66, 50, 49, 66, 49,  2, 23, 10,  2, 35, 63,
        71, 49, 68,  2, 49, 62, 52,  2, 38, 49, 69, 60,  2, 45, 53, 66, 67, 68,
        57, 62, 53,  1,  2,  2, 71, 57, 68, 56,  2, 35, 57, 51, 56, 49, 53, 60,
         2, 38, 63, 67, 68, 63, 62,  2, 49, 62, 52,  2, 40, 53, 50, 53, 51, 51,
        49,  2, 36, 57, 60, 53, 67,  1, 28, 63, 60, 55, 53, 66,  2, 41, 56, 49,
        59, 53, 67, 64, 53, 49, 66, 53,  2, 34, 57, 50, 66, 49, 66, 73,  1, 56,
        68, 68, 64, 67, 19, 11, 11, 67, 56, 49, 59, 53, 67, 64, 53, 49, 66, 53,
        10, 54, 63, 60, 55, 53, 66, 10, 53, 52, 69, 11, 67, 56, 49, 59, 53, 67,
        64, 53, 49, 66, 53, 67,  9, 71, 63, 66, 59, 67, 11, 49,  9, 61, 57, 52,
        

In [ ]:
#Hyperparameters

batch_size = 4
block_size = 8
max_iters = 5000
n_embed=384
eval_interval = 300
eval_iters = 200
n_head=6
n_layer=6
dropout=0.0
learning_rate = 3e-4

In [ ]:
#splitting into train and validation datasets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([23,  2, 35, 57, 52, 67, 69, 61, 61])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
  context = x[:i]
  target = x[i]
  print(f"input is {context} then target is {target}")


input is tensor([], dtype=torch.int64) then target is 23
input is tensor([23]) then target is 2
input is tensor([23,  2]) then target is 35
input is tensor([23,  2, 35]) then target is 57
input is tensor([23,  2, 35, 57]) then target is 52
input is tensor([23,  2, 35, 57, 52]) then target is 67
input is tensor([23,  2, 35, 57, 52, 67]) then target is 69
input is tensor([23,  2, 35, 57, 52, 67, 69]) then target is 61


In [ ]:
torch.manual_seed(1337)

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

@torch.no_grad()

def estimate_loss():
  out = {}
  m.eval()
  for split in ['train','val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = get_batch(split)
      logits,loss= m(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  m.train()
  return out


xb,yb, = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("________________________________________________________________________________")

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"for input is {context.tolist()} the target :{target}")



inputs:
torch.Size([4, 8])
tensor([[66, 61, 57, 49,  8,  2, 60, 63],
        [25, 27,  2,  2, 31, 67,  2, 49],
        [ 2, 71, 56, 49, 68,  2, 54, 63],
        [60, 60,  2, 71, 56, 57, 51, 56]])
targets:
torch.Size([4, 8])
tensor([[61, 57, 49,  8,  2, 60, 63, 63],
        [27,  2,  2, 31, 67,  2, 49, 60],
        [71, 56, 49, 68,  2, 54, 63, 63],
        [60,  2, 71, 56, 57, 51, 56,  2]])
________________________________________________________________________________
for input is [66] the target :61
for input is [66, 61] the target :57
for input is [66, 61, 57] the target :49
for input is [66, 61, 57, 49] the target :8
for input is [66, 61, 57, 49, 8] the target :2
for input is [66, 61, 57, 49, 8, 2] the target :60
for input is [66, 61, 57, 49, 8, 2, 60] the target :63
for input is [66, 61, 57, 49, 8, 2, 60, 63] the target :63
for input is [25] the target :27
for input is [25, 27] the target :2
for input is [25, 27, 2] the target :2
for input is [25, 27, 2, 2] the target :31
for inpu

In [ ]:
import torch.nn as nn

In [ ]:
class Head(nn.Module):
  def __init__(self,head_size):
    super().__init__()
    self.key = nn.Linear(n_embed,head_size,bias=False)
    self.query = nn.Linear(n_embed,head_size,bias=False)
    self.value = nn.Linear(n_embed,head_size,bias=False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)
    wei = q @ k.transpose(-2,-1)*C**-0.5
    wei = wei.masked_fill(self.tril[:T,:T]==0,float('-inf'))
    wei=F.softmax(wei,dim=-1)
    wei = self.dropout(wei)
    v = self.value(x)
    out = wei @ v
    return out

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(num_heads*head_size,n_embed)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    out = torch.cat([h(x) for h in self.heads],dim = -1)
    out = self.dropout(self.proj(out))
    return out

In [ ]:
class FeedForward(nn.Module):

  def __init__(self,n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed,4*n_embed),
        nn.ReLU(),
        nn.Linear(4*n_embed,n_embed),
        nn.Dropout(dropout),
        )

  def forward(self,x):
    return self.net(x)

In [ ]:
class Block(nn.Module):

  def __init__(self,n_embed,n_head):
    super().__init__()
    head_size = n_embed//n_head
    self.sa = MultiHeadAttention(n_head,head_size)
    self.ffwd = FeedForward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)

  def forward(self,x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

In [ ]:
#creating a Bigram
import torch
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,n_embed)
    self.position_embedding_table = nn.Embedding(block_size,n_embed)
    self.blocks = nn.Sequential(*[Block(n_embed,n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embed)
    self.lm_head = nn.Linear(n_embed,vocab_size)


  def forward(self,idx,targets=None):
    B,T = idx.shape

    tok_emb = self.token_embedding_table(idx)  #(B,T,C)
    pos_emb = self.position_embedding_table(torch.arange(T))
    x= tok_emb + pos_emb #(B,T,C)
    x = self.blocks(x)
    x= self.ln_f(x)
    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits,loss


  def generate(self,idx,max_new_tokens):
    #idx is (B,T) array of indices in current context
    for i in range(max_new_tokens):
      idx_cond = idx[:,-block_size:]
      logits,loss = self(idx_cond)
      logits = logits[:,-1,:]
      probs = F.softmax(logits,dim=-1)
      idx_next = torch.multinomial(probs,num_samples = 1) #(B,1)
      idx = torch.cat((idx,idx_next),dim=1) #(B,T+1)
    return idx


m = BigramLanguageModel()
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1,1),dtype = torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([32, 75])
tensor(4.5332, grad_fn=<NllLossBackward0>)
	5)dS'[N=BEr unmM)M
2dU"P:wx
;vsrBHk)n'YCmTkiDxEcHhikU)Tcx)
'G'h(34
-r[)Ie::n,	kjo[;bvv [[[b2mWe=zOu2


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(),learning_rate)
batch_size=32
for steps in range(3000):
  xb,yb = get_batch('train')
  logits,loss = m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

In [ ]:
#Training Loop

for iter in range(max_iters):
  if iter%eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss : {losses['train']:.4f}, val loss : {losses['val']:.4f}")

step 0: train loss : 1.7255, val loss : 1.8865
step 300: train loss : 1.7112, val loss : 1.8761
step 600: train loss : 1.7188, val loss : 1.8781
step 900: train loss : 1.7062, val loss : 1.8777
step 1200: train loss : 1.7225, val loss : 1.8861
step 1500: train loss : 1.7105, val loss : 1.8691
step 1800: train loss : 1.7075, val loss : 1.8764
step 2100: train loss : 1.7129, val loss : 1.8865
step 2400: train loss : 1.7072, val loss : 1.8859
step 2700: train loss : 1.7247, val loss : 1.8657
step 3000: train loss : 1.7244, val loss : 1.8771
step 3300: train loss : 1.7107, val loss : 1.8911
step 3600: train loss : 1.7096, val loss : 1.8772
step 3900: train loss : 1.7290, val loss : 1.8772
step 4200: train loss : 1.7188, val loss : 1.8907
step 4500: train loss : 1.7151, val loss : 1.8782
step 4800: train loss : 1.7066, val loss : 1.8891


In [ ]:
print(decode(m.generate(idx = torch.zeros((1,1),dtype = torch.long),max_new_tokens=500)[0].tolist()))

	Queshey cromollove moonsieur mastern.
	Of Taus and for spurged enca
field, one! BotHere exit.]

I'll play scare your in the come theathing my glance summer's sweet, Lysander me thee man we thou set and gast-see! Master, you dray lay there I do beforth of now you,
	Of lignct with moonloox, gain the for simmetimly, I sweet next prius! That componace maw onlyte to hermsence stolempany very The liates Thisbe heaut, by there.
Tellack,
The angal pasy, the course yeap.
Whate Meter. Limploing with have 
